In [0]:
# PROJECT: predict customer churn
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('logreg_consult').getOrCreate()
df = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/sejal@ibm.com/customer_churn.csv", inferSchema=True, header=True)

In [0]:
df.printSchema()

root
-- Names: string (nullable = true)
-- Age: double (nullable = true)
-- Total_Purchase: double (nullable = true)
-- Account_Manager: integer (nullable = true)
-- Years: double (nullable = true)
-- Num_Sites: double (nullable = true)
-- Onboard_date: string (nullable = true)
-- Location: string (nullable = true)
-- Company: string (nullable = true)
-- Churn: integer (nullable = true)

In [0]:
df.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
 Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites| Onboard_date| Location| Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
 Cameron Williams|42.0| 11066.8| 0| 7.22| 8.0|2013-08-30 07:00:40|10265 Elizabeth M...| Harvey LLC| 1|
 Kevin Mueller|41.0| 11916.22| 0| 6.5| 11.0|2013-08-13 00:38:46|6157 Frank Garden...| Wilson PLC| 1|
 Eric Lozano|38.0| 12884.75| 0| 6.67| 12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...| 1|
 Phillip White|42.0| 8010.76| 0| 6.71| 10.0|2014-04-22 12:43:12|13120 Daniel Moun...| Smith Inc| 1|
 Cynthia Norton|37.0| 9191.58| 0| 5.56| 9.0|2016-01-19 15:31:15|765 Tricia Row Ka...| Love-Jones| 1|
 Jessica Williams|48.0| 10356.02| 0| 5.12| 8.0|2009-03-03 23:13:37|6187 Olson Mounta...| Kelly-Warren| 1|
 Eric Butler|44.0| 11331.58| 1| 5.23| 11.0|2016-12-05 03:35:43|4846 Savannah Roa...| Reynolds-Sheppard| 1|
 Zachary Walsh|32.0| 9885.12| 1| 6.92| 9.0|2006-03-09 14:50:20|25271 Roy Express...| Singh-Cole| 1|
 Ashlee Carr|43.0| 14062.6| 1| 5.46| 11.0|2011-09-29 05:47:23|3725 Caroline Str...| Lopez PLC| 1|
 Jennifer Lynch|40.0| 8066.94| 1| 7.11| 11.0|2006-03-28 15:42:45|363 Sandra Lodge ...| Reed-Martinez| 1|
 Paula Harris|30.0| 11575.37| 1| 5.22| 8.0|2016-11-13 13:13:01|Unit 8120 Box 916...|Briggs, Lamb and ...| 1|
 Bruce Phillips|45.0| 8771.02| 1| 6.64| 11.0|2015-05-28 12:14:03|Unit 1895 Box 094...| Figueroa-Maynard| 1|
 Craig Garner|45.0| 8988.67| 1| 4.84| 11.0|2011-02-16 08:10:47|897 Kelley Overpa...| Abbott-Thompson| 1|
 Nicole Olson|40.0| 8283.32| 1| 5.1| 13.0|2012-11-22 05:35:03|11488 Weaver Cape...|Smith, Kim and Ma...| 1|
 Harold Griffin|41.0| 6569.87| 1| 4.3| 11.0|2015-03-28 02:13:44|1774 Peter Row Ap...|Snyder, Lee and M...| 1|
 James Wright|38.0| 10494.82| 1| 6.81| 12.0|2015-07-22 08:38:40|45408 David Path ...| Sanders-Pierce| 1|
 Doris Wilkins|45.0| 8213.41| 1| 7.35| 11.0|2006-09-03 06:13:55|28216 Wright Moun...|Andrews, Adams an...| 1|
Katherine Carpenter|43.0| 11226.88| 0| 8.08| 12.0|2006-10-22 04:42:38|Unit 4948 Box 481...|Morgan, Phillips ...| 1|
 Lindsay Martin|53.0| 5515.09| 0| 6.85| 8.0|2015-10-07 00:27:10|69203 Crosby Divi...| Villanueva LLC| 1|
 Kathy Curry|46.0| 8046.4| 1| 5.69| 8.0|2014-11-06 23:47:14|9569 Caldwell Cre...|Berry, Orr and Ca...| 1|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
only showing top 20 rows

In [0]:
df.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
summary| Names| Age| Total_Purchase| Account_Manager| Years| Num_Sites| Onboard_date| Location| Company| Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
 count| 900| 900| 900| 900| 900| 900| 900| 900| 900| 900|
 mean| null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777| null| null| null|0.16666666666666666|
 stddev| null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.7648355920350969| null| null| null| 0.3728852122772358|
 min| Aaron King| 22.0| 100.0| 0| 1.0| 3.0|2006-01-02 04:16:13|00103 Jeffrey Cre...| Abbott-Thompson| 0|
 max|Zachary Walsh| 65.0| 18026.01| 1| 9.15| 14.0|2016-12-28 04:07:38|Unit 9800 Box 287...|Zuniga, Clark and...| 1|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+

In [0]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['Age', 'Total_Purchase', 'Account_Manager', 'Years', 'Num_Sites'], 
                           outputCol='features')
output = assembler.transform(df)
final_data = output.select('features', 'churn')
final_data.show()

+--------------------+-----+
 features|churn|
+--------------------+-----+
[42.0,11066.8,0.0...| 1|
[41.0,11916.22,0....| 1|
[38.0,12884.75,0....| 1|
[42.0,8010.76,0.0...| 1|
[37.0,9191.58,0.0...| 1|
[48.0,10356.02,0....| 1|
[44.0,11331.58,1....| 1|
[32.0,9885.12,1.0...| 1|
[43.0,14062.6,1.0...| 1|
[40.0,8066.94,1.0...| 1|
[30.0,11575.37,1....| 1|
[45.0,8771.02,1.0...| 1|
[45.0,8988.67,1.0...| 1|
[40.0,8283.32,1.0...| 1|
[41.0,6569.87,1.0...| 1|
[38.0,10494.82,1....| 1|
[45.0,8213.41,1.0...| 1|
[43.0,11226.88,0....| 1|
[53.0,5515.09,0.0...| 1|
[46.0,8046.4,1.0,...| 1|
+--------------------+-----+
only showing top 20 rows

In [0]:
train_churn, test_churn = final_data.randomSplit([0.7, 0.3])

In [0]:
from pyspark.ml.classification import LogisticRegression
lr_churn = LogisticRegression(labelCol='churn')
fitted_churn_model = lr_churn.fit(train_churn)
training_sum = fitted_churn_model.summary
training_sum.predictions.describe().show()

+-------+------------------+-------------------+
summary| churn| prediction|
+-------+------------------+-------------------+
 count| 589| 589|
 mean|0.1697792869269949|0.13752122241086587|
 stddev|0.3757578992142943| 0.3446895008769917|
 min| 0.0| 0.0|
 max| 1.0| 1.0|
+-------+------------------+-------------------+

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
pred_and_labels = fitted_churn_model.evaluate(test_churn)
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
 features|churn| rawPrediction| probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
[22.0,11254.38,1....| 0|[6.30577563829152...|[0.99817759769997...| 0.0|
[26.0,8787.39,1.0...| 1|[1.45615420111610...|[0.81094376503242...| 0.0|
[27.0,8628.8,1.0,...| 0|[7.33367859782063...|[0.99934725992214...| 0.0|
[29.0,8688.17,1.0...| 1|[3.90188044836935...|[0.98019622972186...| 0.0|
[29.0,9617.59,0.0...| 0|[6.09674714932522...|[0.99775487660915...| 0.0|
[29.0,11274.46,1....| 0|[5.73076354546763...|[0.99676589448636...| 0.0|
[29.0,12711.15,0....| 0|[7.02064259812963...|[0.99910754587904...| 0.0|
[29.0,13255.05,1....| 0|[5.21067101745327...|[0.99457161812417...| 0.0|
[30.0,6744.87,0.0...| 0|[5.07470068658067...|[0.99378589875881...| 0.0|
[30.0,8403.78,1.0...| 0|[7.63768926768912...|[0.99951829135247...| 0.0|
[30.0,11575.37,1....| 1|[5.13195440551048...|[0.99412965602286...| 0.0|
[31.0,8688.21,0.0...| 0|[8.50479869055749...|[0.99979754666944...| 0.0|
[32.0,6367.22,1.0...| 0|[3.94015141316750...|[0.98092563591207...| 0.0|
[32.0,9036.27,0.0...| 0|[0.46145486917849...|[0.61335925474902...| 0.0|
[32.0,9885.12,1.0...| 1|[2.69877970364735...|[0.93695459863371...| 0.0|
[32.0,11715.72,0....| 0|[4.38363903565859...|[0.98767396596628...| 0.0|
[32.0,12479.72,0....| 0|[5.79209184504253...|[0.99695769184585...| 0.0|
[33.0,4711.89,0.0...| 0|[8.02870456306792...|[0.99967413601258...| 0.0|
[33.0,8556.73,0.0...| 0|[4.97212420344518...|[0.99311925760972...| 0.0|
[33.0,10309.71,1....| 0|[7.65453523335335...|[0.99952633441926...| 0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 20 rows

In [0]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='churn')
auc = churn_eval.evaluate(pred_and_labels.predictions)
auc

Out[17]: 0.7865900383141763

In [0]:
# predict on unseen / new data
# fit on all data because we have a new test / heldout set
final_lr_model = lr_churn.fit(final_data)

In [0]:
new_customers = spark.read.csv("dbfs:/FileStore/shared_uploads/sejal@ibm.com/new_customers.csv", inferSchema=True, header=True)

In [0]:
new_customers.printSchema()

root
-- Names: string (nullable = true)
-- Age: double (nullable = true)
-- Total_Purchase: double (nullable = true)
-- Account_Manager: integer (nullable = true)
-- Years: double (nullable = true)
-- Num_Sites: double (nullable = true)
-- Onboard_date: string (nullable = true)
-- Location: string (nullable = true)
-- Company: string (nullable = true)

In [0]:
test_new_customers = assembler.transform(new_customers)

In [0]:
# confirm trnasformed data has a features column
test_new_customers.printSchema()

root
-- Names: string (nullable = true)
-- Age: double (nullable = true)
-- Total_Purchase: double (nullable = true)
-- Account_Manager: integer (nullable = true)
-- Years: double (nullable = true)
-- Num_Sites: double (nullable = true)
-- Onboard_date: string (nullable = true)
-- Location: string (nullable = true)
-- Company: string (nullable = true)
-- features: vector (nullable = true)

In [0]:
final_results = final_lr_model.transform(test_new_customers)
final_results.select('Company', 'prediction').show()

+----------------+----------+
 Company|prediction|
+----------------+----------+
 King Ltd| 0.0|
 Cannon-Benson| 1.0|
Barron-Robertson| 1.0|
 Sexton-Golden| 1.0|
 Wood LLC| 0.0|
 Parks-Robbins| 1.0|
+----------------+----------+

In [0]:
test_new_customers.describe().show()

+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+----------------+
summary| Names| Age| Total_Purchase| Account_Manager| Years| Num_Sites| Onboard_date| Location| Company|
+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+----------------+
 count| 6| 6| 6| 6| 6| 6| 6| 6| 6|
 mean| null|35.166666666666664|7607.156666666667|0.8333333333333334|6.808333333333334|12.333333333333334| null| null| null|
 stddev| null| 15.71517313511584|4346.008232825459| 0.408248290463863|3.708737880555414|3.3862466931200785| null| null| null|
 min|Andrew Mccall| 22.0| 100.0| 0| 1.0| 8.0|2006-12-11 07:48:13|085 Austin Views ...|Barron-Robertson|
 max| Taylor Young| 65.0| 13147.71| 1| 10.0| 15.0|2016-10-28 05:32:13|Unit 0789 Box 073...| Wood LLC|
+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+----------------+